# Deep Image Stack Annotation

![](assets/20240723_deep_image_stack_annotation.png)


## Overview

This workflow is an exension of the primary [deep image stack workflow](./workflow_deep-image-stack.ipynb) that adds annotation capabilities.

### Key Software
Alongside [HoloViz](https://github.com/holoviz), [Bokeh](https://holoviz.org/), and several other foundational packages, we make extensive use of key open source libraries to implement our solution, such as:

- **[HoloNote](https://github.com/pydata/xarray):** Provides for annotation capabilities on HoloViews elements.

## Prerequisites and Resources

| Topic | Type | Notes |
| --- | --- | --- |
| [Deep Image Stack Workflow](./workflow_deep-image-stack.ipynb) | Prerequisite | Primary workflow |

## Imports and Configuration

In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
import holoviews as hv
import panel as pn

pn.extension()
hv.extension('bokeh')

## Loading and Inspecting the Data

In [ ]:
DATA_PATH = "data/real_miniscope_uint8.zarr"

ds = xr.open_dataset(
    DATA_PATH,
    engine = 'zarr',
    chunks = {'frame': 400, 'height':-1, 'width':-1},  # chunk by sets of complete frames
)
da = ds['varr_ref']
da

## Data Visualization

Let's start with our simple HoloViews based app that is complementary to the other approches in the primary workflow. We will want to customize the positioning of the Here, for a straightforward way get a handle on the slider widget that controls the active frame, we'll use a DynamicMap that calls plot_image every time the slider value change.

In [ ]:
# Link function to create image from player widget frame value
def plot_image(value):
    return hv.Image(da.sel(frame=value), kdims=["width", "height"]).opts(
        title=f'frame = {value}',
        frame_height=da.sizes['height'],
        frame_width=da.sizes['width'],
        cmap = "Viridis",
        clim=(0,20),
        colorbar = True,
        tools=['hover', 'crosshair'],
        toolbar='right',
        scalebar=True,
        scalebar_unit=("µm", "m"), # each data bin is about 1 µm
        apply_hard_bounds=True,
        scalebar_opts={
        'background_fill_alpha': 0.5,
        'border_line_color': None,
        'bar_length': 0.10,
        }
    )

video_player = pn.widgets.Player(
    length =len(da.coords["frame"]),
    interval = 100,  # ms
    value = 250, # start frame
    height=90,
    loop_policy="loop",
    align='center',
)

main_view = hv.DynamicMap(pn.bind(plot_image, video_player.param.value_throttled))

# Frame Number Indicator
frame_reactive = pn.bind(lambda value: f'### frame: {value}', video_player.param.value)
frame_markdown = pn.pane.Markdown(object=frame_reactive)

# Max Over Time Overlay
max_proj_time = da.max('frame').compute().astype(np.float32)
img_max_proj_time = hv.Image(
    max_proj_time, ['width', 'height'], label='Max Over Time').opts(
    frame_height=da.sizes['height'],
    frame_width=da.sizes['width'],
    cmap='magma',
)
alpha_slider = pn.widgets.FloatSlider(start=0, end=1, step=.1, value=0.3, name='Alpha of Max Over Time', align='center')
alpha_slider.jslink(img_max_proj_time, value='glyph.global_alpha')

player_layout = pn.WidgetBox('## Playback', video_player, frame_markdown, horizontal=True, align='start')
alpha_slider_layout = pn.WidgetBox('## Max Overlay', alpha_slider, horizontal=True, align='start')

### Spatial Ranges Annotation Extension

We will now construct an annotator object that enables direct annotation of regions of spatial regions interest within the visual interface utilizing the HoloNote package.

In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
from holonote.app.tabulator import AnnotatorTable

annotator = Annotator({"height": float, "width": float}, fields=["type"], groupby="type")
color_dim = hv.dim("type").categorize(
    categories={"A": "red", "B": "orange", "C": "cyan"}, default="grey"
)
annotator.style.color = color_dim
annotator.style.alpha=.5
panel_widgets = PanelWidgets(annotator)
table_widget = AnnotatorTable(annotator)
annotator_widgets = pn.WidgetBox('## Annotator', panel_widgets, table_widget, horizontal=True, align='start', scroll=True)

#### Link Annotations to Timeseries

Finally, we will display the mean timeseries of all the data in the annotated spatial regions.

In [ ]:
# Initialize the empty time series plot
curve_opts = dict(
    responsive=True,
    frame_height=da.sizes['height'],
    min_width=300,
    show_legend=False,
    xlabel='frame',
    tools=['hover'],
    line_alpha=.6,
    framewise=True,
    axiswise=True,
)

def plot_ts(event):
    curves = {}
    df = annotator.df
    for i,row in df.iterrows():
        h1, h2, w1, w2 = row[["start[height]", "end[height]", "start[width]", "end[width]"]]
        da_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        curve = hv.Curve(da_sel.mean(["height", "width"]), group=row['type'], label=f'{i[:6]}')
        curve = curve.opts(subcoordinate_y=True, color=panel_widgets.colormap[row['type']], **curve_opts)
        curves[f"{row['type']} {i[:6]}"] = curve
    time_series.object = (vline * hv.Overlay(curves, kdims=['curve'])).opts(title='Annotated Mean Timeseries', xlim = (frames[0], frames[-1]))

annotator.on_event(plot_ts)

def plot_frame_indicator_line(value):
    if value:
        return hv.VSpans((value, value)).opts(axiswise=True, framewise=True)

line_opts = dict(color="grey", line_width=5, fill_alpha=.5)
vline = hv.DynamicMap(pn.bind(plot_frame_indicator_line, video_player)).opts(**line_opts)

frames = da.coords["frame"].values
time_series = pn.pane.HoloViews(vline * hv.Curve([]).opts(xlim = (frames[0], frames[-1]),
    title='Create an annotation in the image', **curve_opts))

# WIP
# def highlight_selected(indices):
#     print(indices)
#     if indices:
#         # reset all to original alpha
#         for c in time_series.object:
#             c = c.opts(line_alpha=.6)
        
#         # find and change alpha of selected curve
#         i = indices[0]
#         row = annotator.df.loc[i]
#         time_series.object[f"{row['type']} {i[:6]}"] = time_series.object[f"{row['type']} {i[:6]}"].opts(line_color='black')

cached_group_label = set([])

def highlight_last_selected(idx):
    if cached_group_label:
        last_group_label = cached_group_label.pop()
        reset_opts = hv.opts.Curve(last_group_label, line_alpha=0.5)
    else:
        reset_opts = hv.opts.Curve()
    
    # highlight alpha of selected curve
    if len(idx):
        idx = idx[-1]
        row = annotator.df.loc[idx]
        group = f'{row['type']}'
        label = f'{idx[:6]}'
        group_label = f"{group}.{label}"
        cached_group_label.add(group_label)
        ts.object = ts.object.opts(reset_opts, hv.opts.Curve(group_label, line_alpha=1))
    else:
        ts.object = ts.object.opts(reset_opts)

pn.bind(highlight_last_selected, annotator.param.selected_indices, watch=True)

main_view_overlay = main_view * img_max_proj_time * annotator
widgets = pn.WidgetBox(annotator_widgets, alpha_slider_layout, player_layout, align='center')

img_stack_app_annot = pn.Column(
    widgets,
    pn.Row(main_view_overlay, time_series)
)

img_stack_app_annot

### Future work: 
- add annotation selection highlighting to the timeseries
- freehanddraw polygon to box annotation, with polygon saved as a displayable field

In [ ]:
    # if indices:
    #     print(indices[0])
        
    #     # reset all to original alpha
    #     time_series.object.data.opts(line_alpha=.6)
        
    #     # find and change alpha of selected curve
    #     row = annotator.df.loc[indices[0]]
    #     time_series.object.data[f"{row['type']} {i[:6]}"] = time_series.object.data[f"{row['type']} {i[:6]}"].opts(line_alpha=1)


In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
from holonote.app.tabulator import AnnotatorTable
import holoviews as hv; hv.extension('bokeh')
import panel as pn; pn.extension('tabulator')
import numpy as np

annotator = Annotator({"time": float}, fields=["type"], groupby="type")
annotator_widgets = pn.WidgetBox(PanelWidgets(annotator), AnnotatorTable(annotator), horizontal=True)

annotator.set_regions(time=(2,4))
annotator.add_annotation(type='A')
annotator.set_regions(time=(5,6))
annotator.add_annotation(type='B')

curves = {}
for i in range(4):
    curves[f'{i}'] = hv.Curve(np.random.random(10), 'time', 'val', label=f'{i}').opts(subcoordinate_y=True, alpha=.5)

# ts = pn.pane.HoloViews(annotator * hv.Overlay(curves, kdims=['time', 'curve']))

def highlight_selected(indices):
    print('selected indices:', indices)
    # ts.object = ts.object.opts(hv.opts.Curve(line_alpha=.6)) # this resets the selected_indices :(
    row = annotator.df.loc[indices[0]]
    # print(ts.object['2'])
    # ts.object['2'] = ts.object['2'].opts(line_alpha=1)

pn.bind(highlight_selected, annotator.param.selected_indices, watch=True)
# annotator.param.selected_indices.rx.watch(highlight_selected)

pn.Column(annotator_widgets, ts).servable()



In [ ]:
annotator.select_by_index(annotator.df.index[1])

In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets
from holonote.app.tabulator import AnnotatorTable
import holoviews as hv; hv.extension('bokeh')
import panel as pn; pn.extension('tabulator')
import numpy as np
import xarray as xr

# Annotator
annotator = Annotator({"height": float, "width": float}, fields=["type"], groupby="type")
color_dim = hv.dim("type").categorize(
    categories={"A": "red", "B": "orange", "C": "cyan"}, default="grey"
)
annotator.style.color = color_dim
annotator.style.alpha = .5
panel_widgets = PanelWidgets(annotator)
table_widget = AnnotatorTable(annotator)
annotator_widgets = pn.WidgetBox(panel_widgets, table_widget, horizontal=True)

# Data
width, height, frame = 30, 40, 50
frames = np.arange(frame)
data = np.random.random((width, height, frame))

da = xr.DataArray(
    data,
    dims=["width", "height", "frame"],
    coords={
        "width": np.arange(width),
        "height": np.arange(height),
        "frame": frames
    },
    name="data"
)

# App
def sel_frame(iframe):
    return hv.Image(da.sel(frame=iframe)).opts(title='Image')
    
frame_slider = pn.widgets.DiscreteSlider(options=list(frames), value=frames[0])
dyn_img = hv.DynamicMap(pn.bind(sel_frame, frame_slider))

ts = pn.pane.HoloViews(hv.Curve([]).opts(xlim = (frames[0], frames[-1]),
    title='Create an annotation in the image'))

def plot_ts(event):
    curves = {}
    df = annotator.df
    for i, row in df.iterrows():
        group = f'{row['type']}'
        label = f'{i[:6]}'
        h1, h2, w1, w2 = row[["start[height]", "end[height]", "start[width]", "end[width]"]]
        da_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        curve = hv.Curve(da_sel.mean(["height", "width"]), 'frame', group=group, label=label)
        curve = curve.opts(subcoordinate_y=True,
                           color=panel_widgets.colormap[group],
                          line_alpha=.5)
        curves[(group, label)] = curve
    ts.object = (hv.Overlay(curves, kdims=['annotation'])).opts(
        title='Timeseries', xlim = (frames[0], frames[-1]),
        show_legend=False,)

annotator.on_event(plot_ts)

annotator.set_regions(height=(5,15), width=(5,15))
annotator.add_annotation(type='A')
annotator.set_regions(height=(25,30), width=(25,30))
annotator.add_annotation(type='B')

cached_group_label = set([])

def highlight_selected(idx):
    if cached_group_label:
        last_group_label = cached_group_label.pop()
        reset_opts = hv.opts.Curve(last_group_label, line_alpha=0.5)
    else:
        reset_opts = hv.opts.Curve()
    
    # highlight alpha of selected curve
    if len(idx):
        idx = idx[-1]
        row = annotator.df.loc[idx]
        group = f'{row['type']}'
        label = f'{idx[:6]}'
        group_label = f"{group}.{label}"
        cached_group_label.add(group_label)
        ts.object = ts.object.opts(reset_opts, hv.opts.Curve(group_label, line_alpha=1))
    else:
        ts.object = ts.object.opts(reset_opts)

pn.bind(highlight_selected, annotator.param.selected_indices, watch=True)
pn.Column(annotator_widgets, pn.Row(pn.Column(annotator * dyn_img, frame_slider), ts))

In [ ]:
import holoviews as hv; hv.extension('bokeh')
import panel as pn; pn.extension('tabulator')
import numpy as np
import xarray as xr
import pandas as pd

In [ ]:
pd.DataFrame()

In [ ]:
# note, in the real case, we don't know a prior how many curves there will be
df = pd.DataFrame({'A': np.random.random(4)}, index=list(np.arange(4)))
table = pn.widgets.Tabulator(df, disabled=True)

DEFAULT_CURVE_OPTS = hv.opts.Curve(subcoordinate_y=True, line_alpha=0.5)

curves = {}
for i, row in df.iterrows():
    group = f'Group{i}'
    label = f'Label{i}'
    curves[(group, label)] = hv.Curve(np.random.random(10), 'frame', 'val', group=group, label=label).opts(
        DEFAULT_CURVE_OPTS)
plot = hv.Overlay(curves).opts(show_legend=False)
pn_plot = pn.pane.HoloViews(plot)

def show_selected(event):
    idx = df.iloc[event.row].name
    group = f'Group{idx}'
    label = f'Label{idx}'
    hv_obj = pn_plot.object
    hv_obj.opts.clear()
    hv_obj.opts(DEFAULT_CURVE_OPTS)
    pn_plot.object = pn_plot.object.opts(hv.opts.Curve(line_alpha=.5), hv.opts.Curve(f'{group}.{label}', line_alpha=1))
    
table.on_click(show_selected)
pn.Row(table,  pn_plot)

In [ ]:
plot.Label3.opts(line_color='black')

In [ ]:
plot.keys()

In [ ]:
plot.opts(hv.opts.Curve('Group0.Label0', line_color='blue'))

In [ ]:
plot.keys()[0][0]

In [ ]:
plot.opts(hv.opts.Curve(f'.Label{1}', line_alpha=.1))

In [ ]:
table.selection

In [ ]:
import panel as pn
import pandas as pd
import numpy as np
import holoviews as hv

hv.extension("bokeh")

# note, in the real case, we don't know a prior how many curves there will be
df = pd.DataFrame({"A": np.random.random(4)}, index=list(np.arange(4)))
table = pn.widgets.Tabulator(df, disabled=True)

curves = {}
for i, row in df.iterrows():
    group = f"Group{i}"
    label = f"Label{i}"
    curves[(group, label)] = hv.Curve(
        np.random.random(10), "frame", "val", group=group, label=label
    ).opts(subcoordinate_y=True, line_alpha=0.5, )
plot = hv.Overlay(curves).opts(show_legend=False)
pn_plot = pn.pane.HoloViews(plot)


cached_group_label = set([])


def show_selected(event):
    idx = df.iloc[event.row].name
    group = f"Group{idx}"
    label = f"Label{idx}"
    group_label = f"{group}.{label}"
    if cached_group_label:
        last_group_label = cached_group_label.pop()
        reset_opts = hv.opts.Curve(last_group_label, line_alpha=0.5)
    else:
        reset_opts = hv.opts.Curve()
    cached_group_label.add(group_label)
    pn_plot.object = pn_plot.object.opts(
        reset_opts,
        hv.opts.Curve(group_label, line_alpha=1),
    )


table.on_click(show_selected)
pn.Row(table, pn_plot)